# 特異値分解と主成分分析

特異値分解（SVD）と主成分分析（PCA）はデータ解析で重要な手法である。

## 特異値分解（SVD）

特異値分解 (Singular Value Decomposition, SVD) は行列 $A$ を以下のように分解する手法である：

$$
A = U \Sigma V^T
$$

ここで $U$ は直交行列、$\Sigma$ は特異値の対角行列、$V$ は直交行列である。SVD は行列のランクを下げるために利用され、推薦システムでは評価行列の低ランク近似により評価予測を行う：

$$
R_k = U_k \Sigma_k V_k^T
$$

## 主成分分析（PCA）

主成分分析 (Principal Component Analysis, PCA) はデータの次元削減手法である。共分散行列 $C$ の固有値分解を行い、固有ベクトルを利用して次元削減を行う。PCA も SVD を用いて実装可能である。

## 推薦システムにおける応用

推薦システムでは、PCA による特徴抽出や SVD による評価行列の低ランク近似により、ユーザーの好みを予測し、適切なアイテムを推薦する。


### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/linalg/01/01_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/linalg/01/01_nb.ipynb)


### 実行環境
OSはmacOSです。LinuxやUnixのコマンドとはオプションが異なりますので注意。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



以下、特異値分解と主成分分析について、Pythonの実装例を示しつつ、具体的に説明する。

## 特異値分解（SVD）

特異値分解 (Singular Value Decomposition, SVD) は、行列 $A$ を以下のように分解する手法である：

$$
A = U \Sigma V^T
$$

ここで、
- $A$ は $m \times n$ 行列
- $U$ は $m \times m$ の直交行列
- $\Sigma$ は $m \times n$ の対角行列（特異値が対角成分）
- $V$ は $n \times n$ の直交行列

### 計算例

例えば、$3 \times 2$ 行列 $A$ を考える：

$$
A = \begin{pmatrix}
1 & 2 \\
3 & 4 \\
5 & 6
\end{pmatrix}
$$

SVD により、$A$ を以下のように分解できる：

$$
U = \begin{pmatrix}
-0.2298 & 0.8835 & 0.4082 \\
-0.5247 & 0.2408 & -0.8165 \\
-0.8196 & -0.4018 & 0.4082
\end{pmatrix}, \quad
\Sigma = \begin{pmatrix}
9.5255 & 0 \\
0 & 0.5143 \\
0 & 0
\end{pmatrix}, \quad
V = \begin{pmatrix}
-0.6196 & -0.7849 \\
-0.7849 & 0.6196
\end{pmatrix}
$$

### 推薦システムへの応用

推薦システムでは、評価行列 $R$ の低ランク近似に SVD を用いる。例えば、$R$ を次のように分解し、$k$ 個の特異値を使って近似する：

$$
R_k = U_k \Sigma_k V_k^T
$$

この近似により、データのノイズを除去しつつ、評価予測の精度を向上させることができる。

### メリットとデメリット

SVD のメリットは、データの低ランク近似により、データの圧縮やノイズの除去が可能である点である。また、行列のランクを下げることで、計算量を減らすことができる。一方で、SVD は計算コストが高く、大規模データには適用が難しい場合がある。また、特異値の選択方法によっては、情報の損失が発生する可能性がある。



## 主成分分析（PCA）

主成分分析 (Principal Component Analysis, PCA) は、データの次元削減手法である。PCA は以下の手順で行う：

1. データ行列 $X$ の中心化（平均を引く）
2. 共分散行列 $C = \frac{1}{n} X^T X$ の計算
3. 共分散行列 $C$ の固有値分解を行い、固有値と固有ベクトルを求める
4. 固有値の大きい順に固有ベクトルを選び次元削減

### 計算例

例えば、データ行列 $X$ を以下のように定義する：

$$
X = \begin{pmatrix}
2.5 & 2.4 \\
0.5 & 0.7 \\
2.2 & 2.9 \\
1.9 & 2.2 \\
3.1 & 3.0 \\
2.3 & 2.7 \\
2.0 & 1.6 \\
1.0 & 1.1 \\
1.5 & 1.6 \\
1.1 & 0.9
\end{pmatrix}
$$

共分散行列 $C$ を計算すると：

$$
C = \begin{pmatrix}
0.6166 & 0.6154 \\
0.6154 & 0.7166
\end{pmatrix}
$$

この共分散行列の固有値と固有ベクトルを求めると、主成分を得ることができる。

### 推薦システムへの応用

PCA はユーザーやアイテムの特徴を抽出し、次元削減を行うために利用される。例えば、映画推薦システムでは、ユーザーの評価データを主成分分析により特徴ベクトルに変換し、ユーザーの好みに合った映画を推薦することが可能である。

### メリットとデメリット

PCA のメリットは、次元削減によりデータの可視化や計算効率の向上が図れる点である。また、データの分散が最大となる方向を見つけ出すため、データの重要な情報を保持することができる。一方で、PCA は線形性の仮定に依存しており、非線形な関係を持つデータには適用が難しい。また、固有値の選択方法によっては、情報の損失が発生する可能性がある。


In [2]:
# 主成分分析を行うPythonのコード
# ライブラリは使わないで、numpyのみで実装する


import numpy as np

# データの生成
np.random.seed(1)
X = np.random.randn(5, 3)

# データの標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)

# 共分散行列の計算
n_samples = X.shape[0]
cov = np.dot(X.T, X) / n_samples

# 固有値と固有ベクトルの計算
eigenvalues, eigenvectors = np.linalg.eig(cov)

# 主成分の計算
components = np.dot(X, eigenvectors)

# 結果の表示
print("X:")
print(X)
print("共分散行列:")
print(cov)
print("固有値:")
print(eigenvalues)
print("固有ベクトル:")
print(eigenvectors)
print("主成分:")
print(components)

# 結果の確認
# 主成分分析の結果をscikit-learnで確認する
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X)
print("scikit-learnの結果:")
print("主成分:")
print(pca.transform(X))
print("固有ベクトル:")
print(pca.components_)
print("固有値:")
print(pca.explained_variance_)
print("寄与率:")
print(pca.explained_variance_ratio_)
print("累積寄与率:")
print(np.cumsum(pca.explained_variance_ratio_))

X:
[[ 1.13040771 -0.81948067  0.11972492]
 [-1.25266968  0.84821566 -1.21361166]
 [ 1.2368398  -0.98820806  0.75671503]
 [-0.52502048  1.52188148 -1.03211241]
 [-0.58955736 -0.56240842  1.36928412]]
共分散行列:
[[ 1.         -0.73571666  0.46522728]
 [-0.73571666  1.         -0.84323165]
 [ 0.46522728 -0.84323165  1.        ]]
固有値:
[2.37432477 0.541264   0.08441124]
固有ベクトル:
[[-0.52969357  0.77315978  0.34878171]
 [ 0.63119463  0.08462833  0.77099376]
 [-0.56658456 -0.62853958  0.53284138]]
主成分:
[[-1.18385579  0.72938265 -0.17375456]
 [ 1.88653387 -0.13392777 -0.42960179]
 [-1.70764076  0.39701904  0.07269393]
 [ 1.82348234  0.37159307  0.44029139]
 [-0.81851966 -1.36406699  0.09037104]]
scikit-learnの結果:
主成分:
[[-1.18385579 -0.72938265 -0.17375456]
 [ 1.88653387  0.13392777 -0.42960179]
 [-1.70764076 -0.39701904  0.07269393]
 [ 1.82348234 -0.37159307  0.44029139]
 [-0.81851966  1.36406699  0.09037104]]
固有ベクトル:
[[-0.52969357  0.63119463 -0.56658456]
 [-0.77315978 -0.08462833  0.62853958]
 [ 0.

## 結論

本記事では、特異値分解と主成分分析についてPythonの実装を交えつつ解説した。

特異値分解と主成分分析は、データの次元削減や特徴抽出に有効な手法であり、推薦システムにおいても重要な役割を果たす。SVD による評価行列の低ランク近似や PCA による特徴抽出により、ユーザーの好みを予測し、適切なアイテムを推薦することができる。これらの手法を適用する際には、メリットとデメリットを理解し、適切な方法を選択することが重要である。